<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Proxy

## Intro

This pattern provides a surrogate or placeholder to allow control of an object from another place. It's used when we want to provide controlled access to functionality.

The formal definition of a proxy (irrespective of the pattern) is:
> an agent or substitute to a thing with the authority to act as if it were that thing

There are situation in which the client does not or cannot reference the object directly (context boundaries), but still wants to interact with the object. A proxy can provide this indirection and maintain this boundary.

A good real world example of a proxy is a credit card. It is an indirection of the money in a bank account. It behaves as if it is money, but it is in fact not.

There are different types of proxy:
* Remote proxies
    * manages interaction between client and remot object

* Virtual proxy
    * Controls access to the object that is expensive to create

* copy-on-write proxy
    * defers cloning a target until it is actually needed. Kinda a virtual proxy

* Protection access
    * As it sounds, gives different clients different access
    

* Cache proxy
    * Provides temporary storage of the results of expensive target ops

    
* firewall proxy
    * Like non dev, protects targets from bad clients

* Synchronisation proxy
    * provides multiple access routes to a target

The purpose of the pattern is to represent another object and control access to it.

Advantages of the pattern are:
* Added security
* Avoids duplication of objects that carry risk
* Better performance of memory heavy objects

## Implementation

In [1]:
 interface Image {
     void display();
 }

In [3]:
class RealImage implements Image {
    private String filename;
    
    public RealImage(String filename)
    {
        this.filename = filename;
        loadFromDisk(this.filename);
    }
    
    public void display()
    {
        System.out.println("Displaying " + this.filename);
    }
    
    private void loadFromDisk(String filename)
    {
        System.out.println("Loading " + filename);
    }
}

In [4]:
class ImageProxy implements Image {
    private Image image;
    private String filename;
    
    public ImageProxy(String filename)
    {
        this.filename = filename;
    }
    
    public void display()
    {
        if (image == null) {
            image = new RealImage(this.filename);
        }
        
        image.display();
    }
    
}

In [5]:
Image myProxy = new ImageProxy("my file");

In [6]:
myProxy.display();

Loading my file
Displaying my file


Note that the constructor code ran here. This means that the object was not created until it was needed. Now the proxy has access to the created object as it is a member var on itself.

In [8]:
myProxy.display();

Displaying my file


The above variations of the pattern would require us to manipulate how the target being proxied will be called. For example, if we wanted to make a protection access proxy, we'd need to do a permission check inside the method that required a permission check. Also note the use of delegation here. The proxy does nothing but call the real image.

In [9]:
class ImageProxy implements Image {
    private Image image;
    private String filename;
    
    public ImageProxy(String filename)
    {
        this.filename = filename;
    }
    
    public void display()
    {
        // permission if statement would go here and return early.
        /* permissions is a List type
         * if (!auth.user.permissions.contains("admin")) return;
         *
        */
        if (image == null) {
            image = new RealImage(this.filename);
        }
        
        image.display();
    }
    
}

## Example

In [12]:
interface IProxy {
    void checkHosts(String site);
}

In [15]:
class RealProxy implements IProxy {
    private List<String> hosts = new ArrayList<String>();
    
    public RealProxy(List<String> hosts)
    {
        this.hosts = hosts;
    }
    
    public void checkHosts(String site)
    {
        if (this.hosts.contains(site)) {
            System.out.println("Access denied");
            return;
        }
        
        System.out.println("Site not blacklisted");
    }
}

In [16]:
List<String> bannedSites = new ArrayList<String>(Arrays.asList("the hub", "888.com"));

In [17]:
class ProxyProxy implements IProxy {
    private List<String> hosts = new ArrayList<String>();
    private IProxy theRealProxy;
    
    public ProxyProxy(List<String> hosts)
    {
        this.hosts = hosts;
    }
    
    public void checkHosts(String site)
    {
        if (theRealProxy == null) {
            theRealProxy = new RealProxy(hosts);
        }
        
        theRealProxy.checkHosts(site);
    }
}

In [20]:
IProxy libraryProxy = new ProxyProxy(bannedSites);

In [21]:
libraryProxy.checkHosts("the hub");

Access denied


In [22]:
libraryProxy.checkHosts("google.com");

Site not blacklisted
